In [ ]:
import torch
import gpytorch
from gpytorch.kernels import RBFKernel

covar_module = RBFKernel()
theta_t = torch.rand(3,2,requires_grad=True)
X_hat = torch.rand(5,2,requires_grad=False)


def K_θθ(theta_t):
    
    rslt = covar_module(theta_t,theta_t).evaluate()
    
    return rslt
    
def K_θX(theta_t,X_hat):
    
    rslt = covar_module(theta_t,X_hat).evaluate()
    
    return rslt

def get_K_θθ_dθ(theta_t):
    
    jacobs = torch.autograd.functional.jacobian(func=K_θθ,inputs=(theta_t))
    print(f'jacobs[0]{jacobs.shape}')
    ### we must put it in the right shape
    K_θθ_dθ = jacobs.sum(dim=2).transpose(1,2)
    print(f'K_θθ_dθ {K_θθ_dθ.shape}')
    
    return K_θθ_dθ
    

def get_K_θX_dθ(theta_t,X_hat):
    
    jacobs = torch.autograd.functional.jacobian(func=K_θX,inputs=(theta_t,X_hat))
    print(f'jacobs[0]{jacobs[0].shape}')
    ### we must put it in the right shape
    K_θX_dθ = jacobs[0].sum(dim=2).transpose(1,2)
    print(f'K_θX_dθ {K_θX_dθ.shape}')
    
    return K_θX_dθ

def get_K_θθ_dθ2(theta_t):
    
    jacobs = torch.autograd.functional.jacobian(func=get_K_θX_dθ,inputs=(theta_t,theta_t))
    print(f'jacobs[0]{jacobs[0].shape}')
    ### we must put it in the right shape
    K_θθ_dθ2 = jacobs[0].sum(dim=2).sum(dim=2).transpose(1,2) 
    print(f'K_θθ_dθ2 {K_θθ_dθ2.shape}')
    return K_θθ_dθ2

def get_KXX_inv(X_hat):
    
    sigma_n = 0.5
    diag_noise = sigma_n * torch.eye(X_hat.shape[0])
    K_XX = covar_module(X_hat).evaluate() + diag_noise
    return torch.linalg.inv(K_XX)

a = get_K_θX_dθ(theta_t,X_hat)
b = get_KxX_dx(theta_t,X_hat)

print(a.shape,b.shape)

assert ( (a-b) < 1e-5).all()